In [14]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F


In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)


## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [4]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [9]:
# Define data loader
batch_size = 3
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [10]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break


tensor([[ 87., 134.,  58.],
        [ 91.,  88.,  64.],
        [ 69.,  96.,  70.]])
tensor([[119., 133.],
        [ 81., 101.],
        [103., 119.]])


## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [16]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)
list(model.parameters())

Parameter containing:
tensor([[ 0.4023,  0.0356,  0.5234],
        [ 0.3911, -0.1369, -0.2584]], requires_grad=True)
Parameter containing:
tensor([-0.5618,  0.4840], requires_grad=True)


[Parameter containing:
 tensor([[ 0.4023,  0.0356,  0.5234],
         [ 0.3911, -0.1369, -0.2584]], requires_grad=True),
 Parameter containing:
 tensor([-0.5618,  0.4840], requires_grad=True)]

PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [15]:
# Generate predictions
preds = model(inputs)
# preds

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [17]:
# Define loss function
loss_fn = F.mse_loss
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(5039.3540, grad_fn=<MseLossBackward>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [18]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [24]:
def fit(train_dl, model, loss_function, opt, its):

    for _ in range(its):
        # Train with batches of data
        for input, target in train_dl:
        
            pred = model(input)
            loss = loss_function(pred, target)
            loss.backward()

            # 4. Update parameters using gradients
            opt.step()
            # 5. Reset the gradients to zero
            opt.zero_grad()

        if (_+1) % 10 == 0:
            print('It [{}/{}], Loss: {:.4f}'.format(_+1, its, loss.item()))



loss.item returns the actual value stored in the loss tensor.

In [30]:
fit(train_dl, model, loss_fn, opt, 1)

In [49]:
inputs, targets = train_ds[:][0], train_ds[:][1]

print(f'Loss: {loss_fn(model(inputs), targets)}')
print(model(inputs))
print(targets)

Loss: 6.17858362197876
tensor([[ 56.8575,  70.6124],
        [ 81.9544,  98.1307],
        [118.3496, 137.1141],
        [ 20.9604,  38.9311],
        [101.5421, 113.7795],
        [ 55.6160,  69.4865],
        [ 81.7952,  97.8183],
        [118.6417, 137.4874],
        [ 22.2019,  40.0569],
        [102.6245, 114.5930],
        [ 56.6983,  70.3000],
        [ 80.7128,  97.0048],
        [118.5088, 137.4265],
        [ 19.8780,  38.1176],
        [102.7837, 114.9054]], grad_fn=<AddmmBackward>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])
